# Chess Model Training - Qwen2.5-1.5B-Instruct

Fine-tune Qwen2.5-1.5B-Instruct on ~280K chess positions.

**Hardware**: A40/A100 GPU

**Training**: Full fine-tuning

## 1. Setup

In [ ]:
!pip install -q torch transformers datasets accelerate bitsandbytes
!pip install -q trl peft wandb hf_transfer
!pip install -q flash-attn --no-build-isolation

In [ ]:
import os
os.environ["HF_HUB_ENABLE_HF_TRANSFER"] = "1"

import json
import torch
from datetime import datetime
from datasets import Dataset
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from trl import SFTTrainer, SFTConfig
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training

torch.backends.cuda.matmul.allow_tf32 = True
torch.set_float32_matmul_precision("medium")

print(f"PyTorch: {torch.__version__}")
print(f"CUDA: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")

## 2. Configuration

In [ ]:
BASE_MODEL = "Qwen/Qwen2.5-1.5B-Instruct"
OUTPUT_DIR = "./qwen-chess-1.5b-v1"
DATA_PART1 = "train_data_part1.jsonl"
DATA_PART2 = "train_data_part2.jsonl"

TRAINING_MODE = "full"  # "full" or "lora"

CONFIG = {
    "num_epochs": 3,
    "batch_size": 8,
    "gradient_accumulation": 4,
    "learning_rate": 2e-5,
    "warmup_ratio": 0.03,
    "weight_decay": 0.01,
    "max_seq_length": 512,
    "lora_r": 64,
    "lora_alpha": 128,
    "lora_dropout": 0.05,
    "use_flash_attention": True,
    "bf16": True,
    "gradient_checkpointing": True,
    "logging_steps": 100,
    "save_steps": 2000,
    "eval_strategy": "no",
    "dataloader_num_workers": 4,
    "pin_memory": True,
    "optim": "adamw_torch_fused",
}

print(f"Mode: {TRAINING_MODE}")
print(f"Effective batch: {CONFIG['batch_size'] * CONFIG['gradient_accumulation']}")

## 3. Load Data

In [ ]:
def load_jsonl(filepath):
    data = []
    with open(filepath, 'r') as f:
        for line in f:
            data.append(json.loads(line.strip()))
    return data

print("Loading data...")
part1 = load_jsonl(DATA_PART1)
part2 = load_jsonl(DATA_PART2)
all_data = part1 + part2
print(f"Total: {len(all_data):,}")

import random
random.seed(42)
random.shuffle(all_data)

train_size = int(len(all_data) * 0.98)
train_data = all_data[:train_size]
eval_data = all_data[train_size:]
print(f"Train: {len(train_data):,}, Eval: {len(eval_data):,}")

train_dataset = Dataset.from_list(train_data)
eval_dataset = Dataset.from_list(eval_data)

In [ ]:
# Preview
sample = all_data[0]
print("USER:", sample["messages"][0]["content"][:300], "...")
print("\nASSISTANT:", sample["messages"][1]["content"])

## 4. Load Model

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL, trust_remote_code=True)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"
print(f"Tokenizer loaded. Vocab: {tokenizer.vocab_size}")

In [ ]:
model_kwargs = {
    "trust_remote_code": True,
    "torch_dtype": torch.bfloat16,
    "device_map": "auto",
}

if CONFIG["use_flash_attention"]:
    model_kwargs["attn_implementation"] = "flash_attention_2"
    print("Using Flash Attention 2")

print(f"Loading {BASE_MODEL}...")
model = AutoModelForCausalLM.from_pretrained(BASE_MODEL, **model_kwargs)

if CONFIG["gradient_checkpointing"]:
    model.gradient_checkpointing_enable()

if CONFIG["use_flash_attention"]:
    model.config._attn_implementation = "flash_attention_2"
    print(f"Attention impl: {model.config._attn_implementation}")

print(f"Model loaded! Params: {model.num_parameters():,}")

In [ ]:
if TRAINING_MODE == "lora":
    print("Applying LoRA...")
    lora_config = LoraConfig(
        r=CONFIG["lora_r"],
        lora_alpha=CONFIG["lora_alpha"],
        lora_dropout=CONFIG["lora_dropout"],
        bias="none",
        task_type="CAUSAL_LM",
        target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    )
    model = get_peft_model(model, lora_config)
    model.print_trainable_parameters()
else:
    print("Full fine-tuning mode")

## 5. Training

In [ ]:
training_args = SFTConfig(
    output_dir=OUTPUT_DIR,
    num_train_epochs=CONFIG["num_epochs"],
    per_device_train_batch_size=CONFIG["batch_size"],
    per_device_eval_batch_size=CONFIG["batch_size"],
    gradient_accumulation_steps=CONFIG["gradient_accumulation"],
    learning_rate=CONFIG["learning_rate"] if TRAINING_MODE == "full" else 2e-4,
    warmup_ratio=CONFIG["warmup_ratio"],
    weight_decay=CONFIG["weight_decay"],
    bf16=CONFIG["bf16"],
    tf32=True,
    logging_steps=CONFIG["logging_steps"],
    logging_dir=f"{OUTPUT_DIR}/logs",
    save_strategy="steps",
    save_steps=CONFIG["save_steps"],
    save_total_limit=3,
    eval_strategy=CONFIG["eval_strategy"],
    eval_steps=None,
    seed=42,
    report_to="none",
    optim=CONFIG["optim"],
    dataloader_num_workers=CONFIG["dataloader_num_workers"],
    dataloader_pin_memory=CONFIG["pin_memory"],
)
print("Training args configured!")

In [ ]:
trainer = SFTTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    processing_class=tokenizer,
)

print(f"Trainer ready!")
print(f"Train: {len(train_dataset):,}, Eval: {len(eval_dataset):,}")
steps = len(train_dataset) // (CONFIG['batch_size'] * CONFIG['gradient_accumulation']) * CONFIG['num_epochs']
print(f"Total steps: ~{steps:,}")

In [ ]:
print("Starting training...")
trainer.train()
print("Training complete!")

## 6. Save Model

In [ ]:
final_path = f"{OUTPUT_DIR}/final"
print(f"Saving to {final_path}...")

if TRAINING_MODE == "lora":
    model.save_pretrained(final_path)
    tokenizer.save_pretrained(final_path)
    print("Merging LoRA...")
    merged = model.merge_and_unload()
    merged.save_pretrained(f"{OUTPUT_DIR}/merged")
    tokenizer.save_pretrained(f"{OUTPUT_DIR}/merged")
else:
    trainer.save_model(final_path)
    tokenizer.save_pretrained(final_path)

print("Saved!")

## 7. Test

In [ ]:
def test_model(model, tokenizer, fen, legal_moves):
    prompt = f"""You are an expert chess player. Here is the position in FEN format:
{fen}

Legal moves: {legal_moves}

Select the best move. Keep your thinking to 2 sentences or less, then output your chosen move.
Format:
<think>brief thinking (2 sentences max)</think>
<uci_move>your_move</uci_move>"""
    
    messages = [{"role": "user", "content": prompt}]
    text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    inputs = tokenizer(text, return_tensors="pt").to(model.device)
    
    with torch.no_grad():
        outputs = model.generate(**inputs, max_new_tokens=100, temperature=0.1, do_sample=True, pad_token_id=tokenizer.pad_token_id)
    
    return tokenizer.decode(outputs[0][inputs['input_ids'].shape[1]:], skip_special_tokens=True)

# Test
fen = "rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR w KQkq - 0 1"
moves = "e2e4 d2d4 g1f3 b1c3 c2c4"
print("Testing...")
print(test_model(model, tokenizer, fen, moves))

## 8. Upload to HuggingFace (Optional)

In [ ]:
# from huggingface_hub import login
# login()
# model.push_to_hub("your-username/qwen-chess-1.5b")
# tokenizer.push_to_hub("your-username/qwen-chess-1.5b")